In [250]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import display

# TODOs:
# get an error bound based on the data
# analyze trace with full symbols list
# filter out useless symbol (or just bar chart/visualize everything)

attestation = pd.read_csv("data/attestation2.csv")
aes = pd.read_csv("data/tinyaes.csv")
aesfull = pd.read_csv("data/tinyaesfull.csv")
aesfull[:50]

,cycle,env,symbol
0,9224967403,linux,keystone_create_enclave
1,9224967483,linux,calculate_required_pages
2,9224967611,linux,create_enclave
3,9224967930,linux,kmem_cache_alloc
4,9224968603,linux,__get_free_pages
5,9224968714,linux,__alloc_pages_nodemask
6,9224969211,linux,get_page_from_freelist
7,9224970014,linux,__zone_watermark_ok
8,9224970522,linux,_raw_spin_lock_irqsave
9,9224973696,linux,__mod_zone_page_state


In [251]:
class Boundary:
    """A boundary is defined by a start and end symbol along with a name."""
    def __init__(self, name, start, end):
        self.name = name
        self.start = start
        self.end = end

In [252]:
"""Given a set of boundaries (in no order) gather the number of cycles it took
to reach the end of each of the boundaries, by returning a pandas dataframe of
the boundaries, their labels, and the cycle cost. This should work in the case
of multiple enclaves being started, running a user/runtime, and then destroying.
"""

def get_indicies(bset, data):
    """Return the indicies of all the found possible boundaries.
    """
    start_symbols = [b.start for b in bset]
    end_symbols = [b.end for b in bset]
    indicies = []
    for i, row in data.iterrows():
        if row['symbol'] in start_symbols or row['symbol'] in end_symbols:
            if i not in indicies:
                indicies.append(i)
                
    return indicies

# def get_phase_cycles(indicies, bset, data):
#     """Given indicies of interest and the corresponding data
#     return the cycle count of each phase.
#     """
#     start_symbols = [b.start for b in bset]
#     end_symbols = [b.end for b in bset]
    
#     # focus the data
#     df = data.iloc[indicies]
#     # clean up table
#     df["end"] = df["symbol"].shift(-1)
#     df["cost"] = -(df['cycle'] - df['cycle'].shift(-1))
#     df = df.drop(['cycle'], axis=1)[:-1]
#     # boundary matching
#     labels = []
#     for i, row in data.iterrows():
#         start = row['symbol']
#         end = row['end']
#         res = None
#         for b in bset:
#             if b.start == start and b.end == end:
#                 res = b.name
#                 break
#         labels.append(res)
#     print(labels)
#     # append labels in new column/series
#     df["phase"] = pd.Series(labels).values
            
#     return df

def get_phase_cycles2(indicies, bset, data):
    """Given indicies of interest and the corresponding data
    return the cycle count of each phase, along with the 
    start and end indicies of each phase which can be used
    for further analysis
    """
    start_symbols = [b.start for b in bset]
    end_symbols = [b.end for b in bset]
    
    # focus the data
    df = data.iloc[indicies]
    # boundary matching
    phases = []
    cycle_counts = []
    start_indicies = []
    end_indicies = []
    for startidx, row in df.iterrows():
        start = row['symbol']
        # scan everything in front of the current row
        for endidx, next_row in df.loc[startidx:].iterrows():
            end = next_row['symbol']
            print(start, end)
            found_end = False
            for b in bset:
                if b.start == start and b.end == end:
                    total_cycles = next_row['cycle'] - row['cycle'] 
                    phases.append(b.name)
                    cycle_counts.append(total_cycles)
                    start_indicies.append(startidx)
                    end_indicies.append(endidx)
                    found_end = True
                    break # on first one, we can stop
            if found_end:
                break
    labels = ["phases", "cycle_counts", "start_index", "end_index"]
    res = pd.DataFrame([phases, cycle_counts, start_indicies, end_indicies]).transpose()
    res.columns = labels
    return res

# Macro boundaries for the enclave
# We will also want to have smaller microboundaries for the enclave within these larger bucket.
# For these, we will not create separate boundaries.
# With the boundaries within boundaries, we can identify what's slowing down the enclave.
create_bound = Boundary("create_enclave", "keystone_create_enclave", "keystone_run_enclave")
run_bound = Boundary("run_enclave", "keystone_run_enclave", "rt_base")
runtime_bound = Boundary("enclave_runtimeuser", "rt_base", "mcall_sm_attest_enclave")
attest_bound = Boundary("enclave_attestation", "mcall_sm_attest_enclave", "return_to_encl")
return_to_runtime_bound = Boundary("enclave_runtimeuser", "return_to_encl", "mcall_sm_stop_enclave")
attest_edge_call_bound = Boundary("enclave_attestation_edge_call", "mcall_sm_stop_enclave", "return_to_encl")
run_to_exit_bound = Boundary("enclave_runtimeuser", "return_to_encl", "mcall_sm_exit_enclave")
run_wout_attest_bound = Boundary("enclave_runtimeuser", "rt_base","mcall_sm_stop_enclave")
destroy_bound = Boundary("destroy_enclave", "destroy_enclave", "keystone_destroy_enclave_end")
destroy_bound_full = Boundary("destroy_enclave", "destroy_enclave", "_raw_spin_unlock_bh")

enclave_bset = [create_bound, run_bound, runtime_bound, attest_bound, return_to_runtime_bound, attest_edge_call_bound,
                run_to_exit_bound, destroy_bound, destroy_bound_full, run_wout_attest_bound]

native_bset = [] # we just want user mode?
ignore_set = ["printf"]

In [258]:
# Run Analysis
def get_large_buckets(data):
    bset = enclave_bset
    indicies = get_indices(bset, data)
    display(data.iloc[indicies])
    results = get_phase_cycles2(indicies, bset, data)
    return results[['phases', 'cycle_counts']].groupby("phases").sum()

def data_pipeline(data):
    bset = enclave_bset
    indicies = get_indices(bset, data)
    results = get_phase_cycles2(indicies, bset, data)
    return results

In [259]:
aesfullres = data_pipeline(aesfull)

('keystone_create_enclave', 'keystone_create_enclave')
('keystone_create_enclave', '_raw_spin_unlock_bh')
('keystone_create_enclave', 'keystone_run_enclave')
('_raw_spin_unlock_bh', '_raw_spin_unlock_bh')
('_raw_spin_unlock_bh', 'keystone_run_enclave')
('_raw_spin_unlock_bh', '_raw_spin_unlock_bh')
('_raw_spin_unlock_bh', 'rt_base')
('_raw_spin_unlock_bh', 'mcall_sm_stop_enclave')
('_raw_spin_unlock_bh', '_raw_spin_unlock_bh')
('_raw_spin_unlock_bh', 'destroy_enclave')
('_raw_spin_unlock_bh', '_raw_spin_unlock_bh')
('keystone_run_enclave', 'keystone_run_enclave')
('keystone_run_enclave', '_raw_spin_unlock_bh')
('keystone_run_enclave', 'rt_base')
('_raw_spin_unlock_bh', '_raw_spin_unlock_bh')
('_raw_spin_unlock_bh', 'rt_base')
('_raw_spin_unlock_bh', 'mcall_sm_stop_enclave')
('_raw_spin_unlock_bh', '_raw_spin_unlock_bh')
('_raw_spin_unlock_bh', 'destroy_enclave')
('_raw_spin_unlock_bh', '_raw_spin_unlock_bh')
('rt_base', 'rt_base')
('rt_base', 'mcall_sm_stop_enclave')
('mcall_sm_stop_en

In [260]:
aesfullres

,phases,cycle_counts,start_index,end_index
0,create_enclave,45121967,0,4416
1,run_enclave,5046,4416,4432
2,enclave_runtimeuser,213228,4432,87482
3,destroy_enclave,15243,104181,104207


In [263]:
aesres = get_large_buckets(aes)

,cycle,env,symbol
0,5154831588,linux,keystone_create_enclave
110,5199996668,linux,keystone_run_enclave
117,5200001742,runtime,rt_base
83157,5200214564,sm,mcall_sm_stop_enclave
83167,5200536771,linux,destroy_enclave
83169,5200552091,linux,keystone_destroy_enclave_end


('keystone_create_enclave', 'keystone_create_enclave')
('keystone_create_enclave', 'keystone_run_enclave')
('keystone_run_enclave', 'keystone_run_enclave')
('keystone_run_enclave', 'rt_base')
('rt_base', 'rt_base')
('rt_base', 'mcall_sm_stop_enclave')
('mcall_sm_stop_enclave', 'mcall_sm_stop_enclave')
('mcall_sm_stop_enclave', 'destroy_enclave')
('mcall_sm_stop_enclave', 'keystone_destroy_enclave_end')
('destroy_enclave', 'destroy_enclave')
('destroy_enclave', 'keystone_destroy_enclave_end')
('keystone_destroy_enclave_end', 'keystone_destroy_enclave_end')


In [264]:
attres = get_large_buckets(attestation)

,cycle,env,symbol
0,19375570868,linux,keystone_create_enclave
115,19425100177,linux,keystone_run_enclave
122,19425105386,runtime,rt_base
155,19426383019,runtime,return_to_encl
184,19426389773,sm,mcall_sm_attest_enclave
191,19431292572,sm,mcall_sm_exit_enclave
196,19431296470,runtime,return_to_encl
207,19431298222,sm,mcall_sm_stop_enclave
217,19431618647,linux,destroy_enclave
219,19431634197,linux,keystone_destroy_enclave_end


('keystone_create_enclave', 'keystone_create_enclave')
('keystone_create_enclave', 'keystone_run_enclave')
('keystone_run_enclave', 'keystone_run_enclave')
('keystone_run_enclave', 'rt_base')
('rt_base', 'rt_base')
('rt_base', 'return_to_encl')
('rt_base', 'mcall_sm_attest_enclave')
('return_to_encl', 'return_to_encl')
('return_to_encl', 'mcall_sm_attest_enclave')
('return_to_encl', 'mcall_sm_exit_enclave')
('mcall_sm_attest_enclave', 'mcall_sm_attest_enclave')
('mcall_sm_attest_enclave', 'mcall_sm_exit_enclave')
('mcall_sm_attest_enclave', 'return_to_encl')
('mcall_sm_exit_enclave', 'mcall_sm_exit_enclave')
('mcall_sm_exit_enclave', 'return_to_encl')
('mcall_sm_exit_enclave', 'mcall_sm_stop_enclave')
('mcall_sm_exit_enclave', 'destroy_enclave')
('mcall_sm_exit_enclave', 'keystone_destroy_enclave_end')
('return_to_encl', 'return_to_encl')
('return_to_encl', 'mcall_sm_stop_enclave')
('mcall_sm_stop_enclave', 'mcall_sm_stop_enclave')
('mcall_sm_stop_enclave', 'destroy_enclave')
('mcall_s

In [257]:
attres

,cycle_counts
phases,
create_enclave,49529309
destroy_enclave,15550
enclave_attestation,4906697
enclave_runtimeuser,6195692
run_enclave,5209


In [230]:
import plotly.plotly as py
import cufflinks as cf

# You'll need to set API key using:
# plotly.tools.set_credentials_file(username='DemoAccount', api_key='lr1c37zw81')
# Find your api_key here: https://plot.ly/settings/api
# Cufflinks binds plotly to pandas dataframes in IPython notebook.

cf.set_config_file(offline=False, world_readable=True, theme='ggplot')

attres.iplot(kind='bar', barmode='stack')

In [231]:
aesres.iplot(kind='bar', barmode='stack')

In [38]:
df = pd.DataFrame(np.random.rand(10, 4), columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,0.289982,0.525064,0.106397,0.200717
1,0.857107,0.688147,0.068649,0.746845
2,0.278780,0.832076,0.562539,0.529062
3,0.356914,0.174221,0.265477,0.805928
4,0.785437,0.199188,0.371346,0.735639
5,0.715124,0.873299,0.652210,0.285473
6,0.840026,0.456250,0.692743,0.094383
7,0.576376,0.678161,0.362135,0.287692
8,0.859217,0.583189,0.924251,0.029439
9,0.836020,0.510472,0.643956,0.090139


In [40]:
df.iplot(kind='bar', barmode='stack')